In [34]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. 

Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

Наименование вакансии.

Предлагаемую зарплату (отдельно минимальную и максимальную).

Ссылку на саму вакансию.

Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [35]:
def parser_superjob(vacancy):

    vacancy_info = []

    params = {
        'keywords': vacancy,
        'page': ''
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
    }

    url = 'https://www.superjob.ru/vacancy/search/'

    response = requests.get(url, params=params, headers=headers)

    soup = bs(response.text, 'lxml')

    page_block = soup.find('a', {'class': 'f-test-button-1'})

    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())

    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(url, params=params, headers=headers)

    soup = bs(html.text, 'lxml')

    vacancy_items = soup.find_all('div', {'class': 'f-test-vacancy-item'})

    for item in vacancy_items:
        vacancy_info.append(item_superjob(item))

    return vacancy_info

In [36]:
def item_superjob(item):

    vacancy_info = {}

    vacancy_name = item.find('a').getText()

    vacancy_info['vacancy_name'] = vacancy_name

    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})

    if not company_name:
        company_name = item.findParent().find('span', {'class': 'f-test-text-vacancy-item-company-name'}).getText()
    else:
        company_name = company_name.getText()

    vacancy_info['company_name'] = company_name

    vacancy_city = item.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[2].getText().split(',')

    vacancy_info['vacancy_city'] = vacancy_city[0]

    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).getText()

    if not salary or salary == 'По договорённости':
        salary_min = None
        salary_max = None
        salary_currency = None

    else:

        salary_split = salary.replace(u'\xa0', u' ').split(' ')
        salary_currency = salary_split[-1]

        if salary_split[0] == 'до':
            salary_min = None
            salary_max = int(salary_split[1] + salary_split[2])
        elif salary_split[0] == 'от':
            salary_min = int(salary_split[1] + salary_split[2])
            salary_max = None
        else:
            salary_min = int(salary_split[0] + salary_split[1])
            salary_max = int(salary_split[-3] + salary_split[-2])

    vacancy_info['salary_min'] = salary_min
    vacancy_info['salary_max'] = salary_max
    vacancy_info['salary_currency'] = salary_currency

    vacancy_url = item.find_all('a')

    if len(vacancy_url) > 1:
        vacancy_url = vacancy_url[-2]['href']
    else:
        vacancy_url = vacancy_url[0]['href']

    vacancy_info['vacancy_url'] = f'https://www.superjob.ru{vacancy_url}'

    vacancy_info['site'] = 'www.superjob.ru'

    return vacancy_info

In [37]:
def parser_hh(vacancy):

    url = 'https://hh.ru/search/vacancy'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
    }
    
    params = {
        'text': vacancy,
        'search_field': 'name',
        'items_on_page': '100',
        'page': ''
    }
    
    response = requests.get(url, params=params, headers=headers)
    soup = bs(response.text, 'lxml')
    
    vacancy_info = []
    
    page_block = soup.find('div', {'data-qa': 'pager-block'})
    
    if not page_block:
        last_page = '1'
    else:
        last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(url, params=params, headers=headers)
    
    soup = bs(html.text, 'lxml')
    
    vacancy_items = soup.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'vacancy-serp-item'})
    
    for item in vacancy_items:
        vacancy_info.append(item_hh(item))

    return vacancy_info

In [38]:
def item_hh(item):

    vacancy_info = {}

    vacancy_name = item.find('span', {'class': 'resume-search-item__name'}).getText().replace(u'\xa0', u' ')

    vacancy_info['vacancy_name'] = vacancy_name

    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText()

    vacancy_info['company_name'] = company_name

    vacancy_city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText().split(', ')[0]

    vacancy_info['vacancy_city'] = vacancy_city

    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText().replace(u'\xa0', u'')

        salary = re.split(r'\s|-', salary)

        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[-2])

        salary_currency = salary[-1]

    vacancy_info['salary_min'] = salary_min
    vacancy_info['salary_max'] = salary_max
    vacancy_info['salary_currency'] = salary_currency


    vacancy_url = item.find('span', {'class': 'resume-search-item__name'}).find('a')['href']

    vacancy_info['vacancy_url'] = vacancy_url[:vacancy_url.find('?')]

    vacancy_info['site'] = 'hh.ru'

    return vacancy_info

In [39]:
def parser_vacancy(vacancy):
        
    vacancy_info = []
    vacancy_info.extend(parser_hh(vacancy))
    vacancy_info.extend(parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_info)

    return df

In [40]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [41]:
df[:100]

,company_name,salary_currency,salary_max,salary_min,site,vacancy_city,vacancy_name,vacancy_url
0,АО Консист-ОС,None,NaN,NaN,hh.ru,Москва,Разработчик JAVA/Python,https://hh.ru/vacancy/36780861
1,"ПАО «МТС»,IT",None,NaN,NaN,hh.ru,Москва,Python разработчик,https://hh.ru/vacancy/36774402
2,Центр компьютерного обучения СПЕЦИАЛИСТ при М...,руб.,NaN,150000.0,hh.ru,Москва,Преподаватель Python & Data Science и Machine ...,https://hh.ru/vacancy/36437887
3,ООО ЦИФРА,None,NaN,NaN,hh.ru,Москва,Python Senior Developer Team Lead,https://hh.ru/vacancy/36562088
4,ООО Кловер Групп,None,NaN,NaN,hh.ru,Казань,Python developer,https://hh.ru/vacancy/36869475
5,Recruitment Boutique S.M.Art,None,NaN,NaN,hh.ru,Москва,Backend разработчик Python/возможно удаленно (...,https://hh.ru/vacancy/36775655
6,ЗАО Научсофт,None,NaN,NaN,hh.ru,Минск,Middle/Senior Python developer,https://hh.ru/vacancy/36799126
7,Сбербанк для экспертов,None,NaN,NaN,hh.ru,Москва,Python разработчик,https://hh.ru/vacancy/36533045
8,ИП Бутаков Анатолий Николаевич,руб.,80000.0,30000.0,hh.ru,Санкт-Петербург,Преподаватель Python для детей в школу програм...,https://hh.ru/vacancy/36779603
9,Тинькофф,None,NaN,NaN,hh.ru,Москва,Старший инженер-разработчик Python,https://hh.ru/vacancy/36238270
